# Suite: Inventory
Using Library Python Selenium to test the web site

In [10]:
# ####### DEBUG ONLY #######
# ####### Load the "autoreload" extension #######
%load_ext autoreload
# ####### always reload modules marked with "%import" #######
%autoreload 2

import os
import sys

sys.path.append("../config")
sys.path.append("../util")
sys.path.append("../tests")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
# Importing packages
from config import config
from datetime import datetime, timedelta
import json
import logging
import logging.config
import pathlib
from pprint import pprint
import fx_test
from fx_test import TestEngine
import time

from selenium.common.exceptions import StaleElementReferenceException, TimeoutException, ElementClickInterceptedException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.common.keys import Keys

import unittest
from ui_helper import UIHelper

## Variables Declaration

In [12]:
log_config = "../config/logging.conf"

# # Create folder "logs"
# pathlib.Path('../logs').mkdir(exist_ok=True)  
# logging.log_file = datetime.now().strftime('../logs/log_%Y%m%d_%H%M%S.log')
logging.config.fileConfig(log_config, disable_existing_loggers=False)
logger = logging.getLogger(__name__)

suite_name = "suite_inventory"
suite_case = "suite_02"
logger.info("Test Suite: {}".format(suite_name))
logger.info("Suite Case: {}".format(suite_case))

[INFO] Test Suite: suite_inventory
[INFO] Suite Case: suite_02


## Functions Definition

### Main Function Begin
def main(args):

In [13]:
mode = config["web"]["mode"]
base_url = config["web"]["base_url"]
timeout = config["web"]["timeout"]
user_name = config["web"]["user_name"]
password = config["web"]["password"]
company_short_name = config["data"]["company_short_name"]
company_name = config["data"]["company_name"]

if mode == "windows":
    driver_path = "../../../library/drivers/chrome/win/chromedriver.exe"
else:
    driver_path = "../../../library/drivers/chrome/mac/chromedriver"
    
logger.info("Mode: {}".format(mode))
logger.info("Base Url: {}".format(base_url))
logger.info("User Name: {}".format(user_name))

[INFO] Mode: windows
[INFO] Base Url: http://211.25.204.108:30002/
[INFO] User Name: putri.wijaya@adv-fusionex.com


In [14]:
# Get web page
instance = TestEngine(mode="windows", driver_path="../../../library/drivers/chrome/win/chromedriver.exe", timeout=30)
instance.get_driver().get(base_url)

WebDriver created!
Mode: windows
Driver Path: ../../../library/drivers/chrome/win/chromedriver.exe
Timeout: 10


<a id='top'></a>
## Table of contents
- <a href='#data_load'>Load Test Data</a>
- <a href='#login'>Login</a>
- <a href='#switch_company'>Switch Company</a>
- <a href='#items'>Inventory - Items</a>

<a id='login'></a>
### Login
<a href='#top'>top</a>

In [15]:
ui = UIHelper(instance, user_name=user_name, password=password)
ui.login()

<a id='data_load'></a>
### Load Test Data
<a href='#top'>top</a>

In [16]:
# Load sample data
data_path =  f"../data/{suite_case}/inventory.json"
data = ui.load_data(data_path)

items = data["items"]

<a id='switch_company'></a>
### Switch Company
<a href='#top'>top</a>

In [10]:
ui.switch_company(company_name)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=87.0.4280.66)


<a id='items'></a>
### Inventory - Items
<a href='#top'>top</a>

In [17]:
ui.click_side_menu("Items")

In [18]:
for item in items:
    print(item)
    locator = (By.XPATH, f'.//tbody/tr[contains(@class, "row") and .//a[contains(text(), "{item["name"]}")]]')
    elements = instance.find_elements(locator)
    
    if len(elements) > 0:
        logger.info(f"Found: {item['name']}")
        instance.scroll_to_element(elements[0])
        ui.click_edit(elements[0])
    else:
        logger.info(f"Not Found: {item['name']}")
        ui.click_add()
    
    ui.enter_text((By.ID, "name"), item["name"])
#     ui.select_dropdown("Tax", item["tax"])
    ui.enter_text((By.ID, "description"), item["description"])
    instance.scroll_down()
    ui.enter_text((By.ID, "sale_price"), item["sale_price"])
    ui.enter_text((By.ID, "purchase_price"), item["purchase_price"])
    ui.select_dropdown("Category", item["category"])
    ui.select_toggle_button("Enabled", item["enabled"])
    
    
    ui.click_save()
    instance.wait(8)

{'name': 'Printer', 'description': 'Frago Printer', 'sale_price': 400, 'purchase_price': 400, 'category': 'Office Equipment', 'tax': '', 'enabled': 'Yes'}
[INFO] Found: Printer
{'name': 'Chair - Meeting', 'description': '', 'sale_price': 400, 'purchase_price': 400, 'category': 'Furniture & Fittings', 'tax': 'SST (6%)', 'enabled': 'Yes'}
[INFO] Found: Chair - Meeting
{'name': 'Chair - Work Station', 'description': '', 'sale_price': 400, 'purchase_price': 400, 'category': 'Furniture & Fittings', 'tax': '', 'enabled': 'Yes'}
[INFO] Not Found: Chair - Work Station
{'name': 'Table - Meeting Room', 'description': '', 'sale_price': 1000, 'purchase_price': 1000, 'category': 'Furniture & Fittings', 'tax': '', 'enabled': 'Yes'}
[INFO] Not Found: Table - Meeting Room
{'name': 'Table - Work Station', 'description': '', 'sale_price': 700, 'purchase_price': 700, 'category': 'Furniture & Fittings', 'tax': '', 'enabled': 'Yes'}
[INFO] Not Found: Table - Work Station
{'name': 'Water Dispenser', 'descri

In [ ]:
instance.close()